In [4]:
import random
import datetime
from hashlib import md5
import sqlite3

import pandas as pd

In [5]:
# ! test_cases_for_sql_data_connector.db

In [6]:
filename = "test_cases_for_sql_data_connector_new.db"
db = sqlite3.connect(filename)

In [7]:
def generate_ascending_list_of_datetimes(
    k,
    start_date=datetime.date(2020,1,1),
    end_date=datetime.date(2020,12,31)
):
    start_time = datetime.datetime(start_date.year, start_date.month, start_date.day)
    days_between_dates = (end_date - start_date).total_seconds()
    
    datetime_list = [start_time + datetime.timedelta(seconds=random.randrange(days_between_dates)) for i in range(k)]
    datetime_list.sort()
    return datetime_list

generate_ascending_list_of_datetimes(10)

[datetime.datetime(2020, 1, 8, 18, 0, 45),
 datetime.datetime(2020, 2, 4, 14, 55, 58),
 datetime.datetime(2020, 2, 28, 2, 6, 4),
 datetime.datetime(2020, 3, 31, 19, 1, 2),
 datetime.datetime(2020, 4, 11, 12, 8, 39),
 datetime.datetime(2020, 6, 3, 19, 0, 36),
 datetime.datetime(2020, 9, 7, 16, 23, 31),
 datetime.datetime(2020, 10, 14, 3, 14, 50),
 datetime.datetime(2020, 12, 7, 0, 57, 30),
 datetime.datetime(2020, 12, 30, 4, 41, 16)]

In [8]:
k = 120
random.seed(1)

timestamp_list = generate_ascending_list_of_datetimes(k, end_date=datetime.date(2020,1,31))
date_list = [datetime.date(ts.year, ts.month, ts.day) for ts in timestamp_list]

batch_ids = [random.randint(0,10) for i in range(k)]
batch_ids.sort()

session_ids = [random.randint(2,60) for i in range(k)]
session_ids.sort()
session_ids = [i-random.randint(0,2) for i in session_ids]

events_df = pd.DataFrame({
    "id" : range(k),
    "batch_id" : batch_ids,
    "date" : date_list,
    "y" : [d.year for d in date_list],
    "m" : [d.month for d in date_list],
    "d" : [d.day for d in date_list],
    "timestamp" : timestamp_list,
    "session_id" : session_ids,
    "event_type" : [random.choice(["start", "stop", "continue"]) for i in range(k)],
    "favorite_color" : ["#"+"".join([random.choice(list("0123456789ABCDEF")) for j in range(6)]) for i in range(k)]
})

# NOTE: in this fake example, id, batch_id, date, and timestamp are all sorted in strictly the same order
# This would not necessarily be true in the real world.
# Events might arrive and be indexed out of order.
# Batches might not correspond strictly with units in time.
# etc.

# events_df.to_sql("events_df", db)

In [9]:
events_df[["id", "date", "event_type", "favorite_color"]].to_sql("table_partitioned_by_date_column__A", db)

In [10]:
events_df[["id", "timestamp", "event_type", "favorite_color"]].to_sql("table_partitioned_by_timestamp_column__B", db)

In [11]:
df = events_df[["id", "event_type", "favorite_color"]].to_sql("table_partitioned_by_regularly_spaced_incrementing_id_column__C", db)

In [12]:
events_df[["id", "event_type", "favorite_color"]].to_sql("table_partitioned_by_irregularly_spaced_incrementing_id_with_spacing_in_a_second_table__D", db)
lookup_df = events_df.groupby("date").id.min()
lookup_df.to_sql("table_containing_id_spacers_for_D", db)

In [13]:
events_df[["id", "batch_id", "event_type", "favorite_color"]].to_sql("table_partitioned_by_incrementing_batch_id__E", db)

In [14]:
events_df[["id", "session_id", "event_type", "favorite_color"]].to_sql("table_partitioned_by_foreign_key__F", db)
sessions_df = events_df.groupby("date").id.min()
sessions_df.to_sql("table_with_fk_reference_from_F", db)

In [15]:
events_df[["id", "y", "m", "d", "event_type", "favorite_color"]].to_sql("table_partitioned_by_multiple_columns__G", db)

In [16]:
events_df[["id", "event_type", "favorite_color"]].to_sql("table_that_should_be_partitioned_by_random_hash__H", db)

In [17]:
events_df.to_sql("table_full__I", db)

## Add Views

In [18]:
db_cursor = db.cursor()

In [19]:
db_cursor.execute("CREATE VIEW view_by_date_column__A AS SELECT id, date, event_type, favorite_color FROM table_full__I")

In [20]:
db_cursor.execute("CREATE VIEW view_by_timestamp_column__B AS SELECT id, timestamp, event_type, favorite_color FROM table_full__I")

In [21]:
db_cursor.execute("CREATE VIEW view_by_regularly_spaced_incrementing_id_column__C AS SELECT id, event_type, favorite_color FROM table_full__I")

In [22]:
db_cursor.execute("CREATE VIEW view_by_irregularly_spaced_incrementing_id_with_spacing_in_a_second_table__D AS SELECT id, event_type, favorite_color FROM table_full__I")

In [23]:
db_cursor.execute("CREATE VIEW view_containing_id_spacers_for_D AS SELECT id, batch_id, event_type, favorite_color FROM table_full__I")

In [24]:
db_cursor.execute("CREATE VIEW view_by_incrementing_batch_id__E AS SELECT id, batch_id, event_type, favorite_color FROM table_full__I")

In [25]:
db_cursor.execute("CREATE VIEW view_partitioned_by_foreign_key__F AS SELECT id, session_id, event_type, favorite_color FROM table_full__I")

In [26]:
db_cursor.execute("CREATE VIEW view_with_fk_reference_from_F AS SELECT id, date FROM table_full__I GROUP BY date")

In [27]:
db_cursor.execute("CREATE VIEW view_by_multiple_columns__G AS SELECT id, y, m, d, event_type, favorite_color FROM table_full__I")

In [28]:
db_cursor.execute("CREATE VIEW view_that_should_be_partitioned_by_random_hash__H AS SELECT id, event_type, favorite_color FROM table_full__I")

## Test if Views Work

In [29]:
db_cursor.execute("SELECT * FROM view_by_date_column__A;")
rows = db_cursor.fetchall()

for row in rows: 
    print(row) 

(0, '2020-01-01', 'start', '#23C548')
(1, '2020-01-01', 'start', '#D61FCB')
(2, '2020-01-01', 'start', '#C51283')
(3, '2020-01-01', 'start', '#8242E7')
(4, '2020-01-02', 'stop', '#CDC5AE')
(5, '2020-01-02', 'stop', '#4F63DD')
(6, '2020-01-02', 'start', '#3987C0')
(7, '2020-01-02', 'continue', '#6E0078')
(8, '2020-01-02', 'continue', '#659468')
(9, '2020-01-02', 'continue', '#98E5BF')
(10, '2020-01-02', 'stop', '#D36C69')
(11, '2020-01-02', 'start', '#303094')
(12, '2020-01-03', 'start', '#2B9DBA')
(13, '2020-01-03', 'start', '#03EEB9')
(14, '2020-01-04', 'continue', '#CAF3CC')
(15, '2020-01-05', 'continue', '#6086ED')
(16, '2020-01-05', 'start', '#95E6B0')
(17, '2020-01-05', 'stop', '#CDCA2F')
(18, '2020-01-05', 'continue', '#790D4C')
(19, '2020-01-06', 'continue', '#8520B8')
(20, '2020-01-06', 'continue', '#D94E8F')
(21, '2020-01-06', 'continue', '#5E183D')
(22, '2020-01-07', 'continue', '#2B2E05')
(23, '2020-01-07', 'start', '#52C896')
(24, '2020-01-08', 'start', '#67A822')
(25, '202

In [30]:
db_cursor.execute("SELECT * FROM view_by_timestamp_column__B;")
rows = db_cursor.fetchall()
for row in rows: 
    print(row) 

(0, '2020-01-01 02:27:13', 'start', '#23C548')
(1, '2020-01-01 06:39:05', 'start', '#D61FCB')
(2, '2020-01-01 10:43:28', 'start', '#C51283')
(3, '2020-01-01 14:19:56', 'start', '#8242E7')
(4, '2020-01-02 01:02:14', 'stop', '#CDC5AE')
(5, '2020-01-02 02:00:25', 'stop', '#4F63DD')
(6, '2020-01-02 05:38:47', 'start', '#3987C0')
(7, '2020-01-02 09:01:37', 'continue', '#6E0078')
(8, '2020-01-02 09:50:05', 'continue', '#659468')
(9, '2020-01-02 10:27:26', 'continue', '#98E5BF')
(10, '2020-01-02 11:38:23', 'stop', '#D36C69')
(11, '2020-01-02 16:13:31', 'start', '#303094')
(12, '2020-01-03 02:39:58', 'start', '#2B9DBA')
(13, '2020-01-03 17:23:18', 'start', '#03EEB9')
(14, '2020-01-04 01:31:29', 'continue', '#CAF3CC')
(15, '2020-01-05 04:44:31', 'continue', '#6086ED')
(16, '2020-01-05 13:21:15', 'start', '#95E6B0')
(17, '2020-01-05 20:30:31', 'stop', '#CDCA2F')
(18, '2020-01-05 23:06:11', 'continue', '#790D4C')
(19, '2020-01-06 05:44:56', 'continue', '#8520B8')
(20, '2020-01-06 17:23:06', 'cont

In [31]:
db_cursor.execute("SELECT * FROM view_by_regularly_spaced_incrementing_id_column__C;")
rows = db_cursor.fetchall()
for row in rows: 
    print(row) 

(0, 'start', '#23C548')
(1, 'start', '#D61FCB')
(2, 'start', '#C51283')
(3, 'start', '#8242E7')
(4, 'stop', '#CDC5AE')
(5, 'stop', '#4F63DD')
(6, 'start', '#3987C0')
(7, 'continue', '#6E0078')
(8, 'continue', '#659468')
(9, 'continue', '#98E5BF')
(10, 'stop', '#D36C69')
(11, 'start', '#303094')
(12, 'start', '#2B9DBA')
(13, 'start', '#03EEB9')
(14, 'continue', '#CAF3CC')
(15, 'continue', '#6086ED')
(16, 'start', '#95E6B0')
(17, 'stop', '#CDCA2F')
(18, 'continue', '#790D4C')
(19, 'continue', '#8520B8')
(20, 'continue', '#D94E8F')
(21, 'continue', '#5E183D')
(22, 'continue', '#2B2E05')
(23, 'start', '#52C896')
(24, 'start', '#67A822')
(25, 'stop', '#BE1598')
(26, 'stop', '#B7CC5F')
(27, 'continue', '#8A7870')
(28, 'start', '#CAD786')
(29, 'start', '#25E48E')
(30, 'continue', '#544EB9')
(31, 'continue', '#C73692')
(32, 'start', '#37CAF3')
(33, 'stop', '#511061')
(34, 'start', '#FEA835')
(35, 'continue', '#37C7FE')
(36, 'stop', '#C5779E')
(37, 'stop', '#C6E8AF')
(38, 'continue', '#362100')

In [32]:
db_cursor.execute("SELECT * FROM view_by_regularly_spaced_incrementing_id_column__C;")
rows = db_cursor.fetchall()
for row in rows: 
    print(row) 

(0, 'start', '#23C548')
(1, 'start', '#D61FCB')
(2, 'start', '#C51283')
(3, 'start', '#8242E7')
(4, 'stop', '#CDC5AE')
(5, 'stop', '#4F63DD')
(6, 'start', '#3987C0')
(7, 'continue', '#6E0078')
(8, 'continue', '#659468')
(9, 'continue', '#98E5BF')
(10, 'stop', '#D36C69')
(11, 'start', '#303094')
(12, 'start', '#2B9DBA')
(13, 'start', '#03EEB9')
(14, 'continue', '#CAF3CC')
(15, 'continue', '#6086ED')
(16, 'start', '#95E6B0')
(17, 'stop', '#CDCA2F')
(18, 'continue', '#790D4C')
(19, 'continue', '#8520B8')
(20, 'continue', '#D94E8F')
(21, 'continue', '#5E183D')
(22, 'continue', '#2B2E05')
(23, 'start', '#52C896')
(24, 'start', '#67A822')
(25, 'stop', '#BE1598')
(26, 'stop', '#B7CC5F')
(27, 'continue', '#8A7870')
(28, 'start', '#CAD786')
(29, 'start', '#25E48E')
(30, 'continue', '#544EB9')
(31, 'continue', '#C73692')
(32, 'start', '#37CAF3')
(33, 'stop', '#511061')
(34, 'start', '#FEA835')
(35, 'continue', '#37C7FE')
(36, 'stop', '#C5779E')
(37, 'stop', '#C6E8AF')
(38, 'continue', '#362100')

In [33]:
db_cursor.execute("SELECT * FROM view_by_irregularly_spaced_incrementing_id_with_spacing_in_a_second_table__D;")
rows = db_cursor.fetchall()
for row in rows: 
    print(row) 

(0, 'start', '#23C548')
(1, 'start', '#D61FCB')
(2, 'start', '#C51283')
(3, 'start', '#8242E7')
(4, 'stop', '#CDC5AE')
(5, 'stop', '#4F63DD')
(6, 'start', '#3987C0')
(7, 'continue', '#6E0078')
(8, 'continue', '#659468')
(9, 'continue', '#98E5BF')
(10, 'stop', '#D36C69')
(11, 'start', '#303094')
(12, 'start', '#2B9DBA')
(13, 'start', '#03EEB9')
(14, 'continue', '#CAF3CC')
(15, 'continue', '#6086ED')
(16, 'start', '#95E6B0')
(17, 'stop', '#CDCA2F')
(18, 'continue', '#790D4C')
(19, 'continue', '#8520B8')
(20, 'continue', '#D94E8F')
(21, 'continue', '#5E183D')
(22, 'continue', '#2B2E05')
(23, 'start', '#52C896')
(24, 'start', '#67A822')
(25, 'stop', '#BE1598')
(26, 'stop', '#B7CC5F')
(27, 'continue', '#8A7870')
(28, 'start', '#CAD786')
(29, 'start', '#25E48E')
(30, 'continue', '#544EB9')
(31, 'continue', '#C73692')
(32, 'start', '#37CAF3')
(33, 'stop', '#511061')
(34, 'start', '#FEA835')
(35, 'continue', '#37C7FE')
(36, 'stop', '#C5779E')
(37, 'stop', '#C6E8AF')
(38, 'continue', '#362100')

In [34]:
db_cursor.execute("SELECT * FROM view_by_incrementing_batch_id__E;")
rows = db_cursor.fetchall()
for row in rows: 
    print(row) 

(0, 0, 'start', '#23C548')
(1, 0, 'start', '#D61FCB')
(2, 0, 'start', '#C51283')
(3, 0, 'start', '#8242E7')
(4, 0, 'stop', '#CDC5AE')
(5, 0, 'stop', '#4F63DD')
(6, 0, 'start', '#3987C0')
(7, 0, 'continue', '#6E0078')
(8, 0, 'continue', '#659468')
(9, 0, 'continue', '#98E5BF')
(10, 0, 'stop', '#D36C69')
(11, 0, 'start', '#303094')
(12, 0, 'start', '#2B9DBA')
(13, 0, 'start', '#03EEB9')
(14, 0, 'continue', '#CAF3CC')
(15, 1, 'continue', '#6086ED')
(16, 1, 'start', '#95E6B0')
(17, 1, 'stop', '#CDCA2F')
(18, 1, 'continue', '#790D4C')
(19, 1, 'continue', '#8520B8')
(20, 1, 'continue', '#D94E8F')
(21, 1, 'continue', '#5E183D')
(22, 1, 'continue', '#2B2E05')
(23, 1, 'start', '#52C896')
(24, 2, 'start', '#67A822')
(25, 2, 'stop', '#BE1598')
(26, 2, 'stop', '#B7CC5F')
(27, 2, 'continue', '#8A7870')
(28, 2, 'start', '#CAD786')
(29, 2, 'start', '#25E48E')
(30, 2, 'continue', '#544EB9')
(31, 2, 'continue', '#C73692')
(32, 2, 'start', '#37CAF3')
(33, 2, 'stop', '#511061')
(34, 2, 'start', '#FEA835'

In [35]:
db_cursor.execute("SELECT * FROM view_partitioned_by_foreign_key__F;")
rows = db_cursor.fetchall()
for row in rows: 
    print(row) 

(0, 3, 'start', '#23C548')
(1, 2, 'start', '#D61FCB')
(2, 4, 'start', '#C51283')
(3, 2, 'start', '#8242E7')
(4, 4, 'stop', '#CDC5AE')
(5, 6, 'stop', '#4F63DD')
(6, 6, 'start', '#3987C0')
(7, 6, 'continue', '#6E0078')
(8, 7, 'continue', '#659468')
(9, 8, 'continue', '#98E5BF')
(10, 8, 'stop', '#D36C69')
(11, 8, 'start', '#303094')
(12, 6, 'start', '#2B9DBA')
(13, 7, 'start', '#03EEB9')
(14, 9, 'continue', '#CAF3CC')
(15, 9, 'continue', '#6086ED')
(16, 8, 'start', '#95E6B0')
(17, 10, 'stop', '#CDCA2F')
(18, 8, 'continue', '#790D4C')
(19, 10, 'continue', '#8520B8')
(20, 10, 'continue', '#D94E8F')
(21, 9, 'continue', '#5E183D')
(22, 12, 'continue', '#2B2E05')
(23, 11, 'start', '#52C896')
(24, 11, 'start', '#67A822')
(25, 12, 'stop', '#BE1598')
(26, 13, 'stop', '#B7CC5F')
(27, 12, 'continue', '#8A7870')
(28, 14, 'start', '#CAD786')
(29, 13, 'start', '#25E48E')
(30, 14, 'continue', '#544EB9')
(31, 15, 'continue', '#C73692')
(32, 17, 'start', '#37CAF3')
(33, 17, 'stop', '#511061')
(34, 16, 's

In [36]:
db_cursor.execute("SELECT * FROM view_with_fk_reference_from_F;")
rows = db_cursor.fetchall()
for row in rows: 
    print(row) 

(0, '2020-01-01')
(4, '2020-01-02')
(12, '2020-01-03')
(14, '2020-01-04')
(15, '2020-01-05')
(19, '2020-01-06')
(22, '2020-01-07')
(24, '2020-01-08')
(25, '2020-01-09')
(28, '2020-01-10')
(33, '2020-01-11')
(37, '2020-01-12')
(43, '2020-01-13')
(45, '2020-01-14')
(47, '2020-01-15')
(51, '2020-01-16')
(52, '2020-01-17')
(55, '2020-01-18')
(59, '2020-01-19')
(64, '2020-01-20')
(69, '2020-01-21')
(74, '2020-01-22')
(79, '2020-01-23')
(83, '2020-01-24')
(87, '2020-01-25')
(97, '2020-01-26')
(100, '2020-01-27')
(107, '2020-01-28')
(111, '2020-01-29')
(117, '2020-01-30')


In [37]:
db_cursor.execute("SELECT * FROM view_by_multiple_columns__G;")
rows = db_cursor.fetchall()
for row in rows: 
    print(row) 

(0, 2020, 1, 1, 'start', '#23C548')
(1, 2020, 1, 1, 'start', '#D61FCB')
(2, 2020, 1, 1, 'start', '#C51283')
(3, 2020, 1, 1, 'start', '#8242E7')
(4, 2020, 1, 2, 'stop', '#CDC5AE')
(5, 2020, 1, 2, 'stop', '#4F63DD')
(6, 2020, 1, 2, 'start', '#3987C0')
(7, 2020, 1, 2, 'continue', '#6E0078')
(8, 2020, 1, 2, 'continue', '#659468')
(9, 2020, 1, 2, 'continue', '#98E5BF')
(10, 2020, 1, 2, 'stop', '#D36C69')
(11, 2020, 1, 2, 'start', '#303094')
(12, 2020, 1, 3, 'start', '#2B9DBA')
(13, 2020, 1, 3, 'start', '#03EEB9')
(14, 2020, 1, 4, 'continue', '#CAF3CC')
(15, 2020, 1, 5, 'continue', '#6086ED')
(16, 2020, 1, 5, 'start', '#95E6B0')
(17, 2020, 1, 5, 'stop', '#CDCA2F')
(18, 2020, 1, 5, 'continue', '#790D4C')
(19, 2020, 1, 6, 'continue', '#8520B8')
(20, 2020, 1, 6, 'continue', '#D94E8F')
(21, 2020, 1, 6, 'continue', '#5E183D')
(22, 2020, 1, 7, 'continue', '#2B2E05')
(23, 2020, 1, 7, 'start', '#52C896')
(24, 2020, 1, 8, 'start', '#67A822')
(25, 2020, 1, 9, 'stop', '#BE1598')
(26, 2020, 1, 9, 'stop'

In [38]:
db_cursor.execute("SELECT * FROM view_that_should_be_partitioned_by_random_hash__H;")
rows = db_cursor.fetchall()
for row in rows: 
    print(row) 

(0, 'start', '#23C548')
(1, 'start', '#D61FCB')
(2, 'start', '#C51283')
(3, 'start', '#8242E7')
(4, 'stop', '#CDC5AE')
(5, 'stop', '#4F63DD')
(6, 'start', '#3987C0')
(7, 'continue', '#6E0078')
(8, 'continue', '#659468')
(9, 'continue', '#98E5BF')
(10, 'stop', '#D36C69')
(11, 'start', '#303094')
(12, 'start', '#2B9DBA')
(13, 'start', '#03EEB9')
(14, 'continue', '#CAF3CC')
(15, 'continue', '#6086ED')
(16, 'start', '#95E6B0')
(17, 'stop', '#CDCA2F')
(18, 'continue', '#790D4C')
(19, 'continue', '#8520B8')
(20, 'continue', '#D94E8F')
(21, 'continue', '#5E183D')
(22, 'continue', '#2B2E05')
(23, 'start', '#52C896')
(24, 'start', '#67A822')
(25, 'stop', '#BE1598')
(26, 'stop', '#B7CC5F')
(27, 'continue', '#8A7870')
(28, 'start', '#CAD786')
(29, 'start', '#25E48E')
(30, 'continue', '#544EB9')
(31, 'continue', '#C73692')
(32, 'start', '#37CAF3')
(33, 'stop', '#511061')
(34, 'start', '#FEA835')
(35, 'continue', '#37C7FE')
(36, 'stop', '#C5779E')
(37, 'stop', '#C6E8AF')
(38, 'continue', '#362100')